<a href="https://colab.research.google.com/github/csaybar/EarthEngineMasterGIS/blob/master/module05/02_metadatos_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!--COURSE_INFORMATION-->
<img align="left" style="padding-right:10px;" src="https://user-images.githubusercontent.com/16768318/73986808-75b3ca00-4936-11ea-90f1-3a6c352766ce.png" width=10% >
<img align="right" style="padding-left:10px;" src="https://user-images.githubusercontent.com/16768318/73986811-764c6080-4936-11ea-9653-a3eacc47caed.png" width=10% >

**Bienvenidos!** Este *colab notebook* es parte del curso [**Introduccion a Google Earth Engine con Python**](https://github.com/csaybar/EarthEngineMasterGIS) desarrollado por el equipo [**MasterGIS**](https://www.mastergis.com/). Obten mas informacion del curso en este [**enlace**](https://www.mastergis.com/product/google-earth-engine/). El contenido del curso esta disponible en [**GitHub**](https://github.com/csaybar/EarthEngineMasterGIS) bajo licencia [**MIT**](https://opensource.org/licenses/MIT).

## **MASTERGIS: Exploración de metadatos II**
## **ee.Geometry, ee.Feature y ee.FeatureCollection**

En esta lectura, aprenderemos sobre:

- Como obtener metadatos en ee.Geometry.
- Como obtener metadatos en ee.Feature.
- Como obtener metadatos en ee.FeatureCollection.

### **1) Autenticar y inicializar GEE**

In [0]:
#@title Credenciales Google Earth Engine
import os 
credential = '{"refresh_token":"PON_AQUI_TU_TOKEN"}'
credential_file_path = os.path.expanduser("~/.config/earthengine/")
os.makedirs(credential_file_path,exist_ok=True)
with open(credential_file_path + 'credentials', 'w') as file:
    file.write(credential)

In [0]:
import ee
ee.Initialize()

### **2) Carga nuestra funcion de mapeo**

In [0]:
#@title mapdisplay: Crea mapas interactivos usando folium
import folium
def mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    center = center[::-1]
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

### **3) Metadatos ee.Geometry y ee.Feature**

Para ver la informacion sobre una geometria, solo utilize **.getInfo**. Para acceder a la informacion de forma programada. Earth Engine proporciona varios metodos:




In [0]:
polygon = ee.Geometry.Polygon([[[-5, 40], [65, 40], [65, 60], [-5, 60], [-5, 60]]])

In [0]:
print('Polygon printout: ', polygon.getInfo())

# Imprime el area del poligono en kilometros cuadrados.
print('Polygon area: ', polygon.area().divide(1000 * 1000).getInfo())

# Imprimir el perimetro del poligono en kilometros.
print('Polygon perimeter: ', polygon.perimeter().divide(1000).getInfo())

# Imprime la geometria como una cadena GeoJSON.
print('Polygon GeoJSON: ', polygon.toGeoJSONString())

# Imprima el 'tipo' GeoJSON
print('Geometry type: ', polygon.type().getInfo())

# Imprimir las coordenadas como listas
print('Polygon coordinates: ', polygon.coordinates().getInfo())

Polygon GeoJSON:  {"type": "Polygon", "coordinates": [[[-5, 40], [65, 40], [65, 60], [-5, 60], [-5, 60]]], "evenOdd": true}


In [0]:
ee_demop = ee.Feature(polygon)
ee_demop = ee_demop.set('area2',10000)
ee_demop.get('area2').getInfo()

10000

### **4) Metadatos ee.FeatureCollection**


In [0]:
from pprint import pprint

# Cargar los datos de cuencas
sheds = ee.FeatureCollection('USGS/WBD/2017/HUC06')

# Verifique el numero de cuencas hidrograficas despues de filtrar por ubicacion.
print('Count after filter:', sheds.size().getInfo())

print('Primer feature: \n')
pprint(sheds.first().toDictionary().getInfo())

Count after filter: 387
Primer feature: 

{'areaacres': '17935746.85',
 'areasqkm': '72583.46',
 'gnis_id': '',
 'huc6': '190301',
 'loaddate': '20120611075458',
 'metasource': '',
 'name': 'Aleutian Islands',
 'shape_area': '9.9308571861444',
 'shape_leng': '81.5083456556245',
 'sourcedata': '',
 'sourcefeat': '',
 'sourceorig': '',
 'states': 'AK',
 'tnmid': '{7D8E9EC1-6A0D-4FED-8063-94688560AC75}'}


In [0]:
ee.Feature(sheds.toList(2).get(1)).toDictionary().getInfo()

### **¿Dudas con este Jupyer-Notebook?**

Estaremos felices de ayudarte!. Create una cuenta Github si es que no la tienes, luego detalla tu problema ampliamente en: https://github.com/csaybar/EarthEngineMasterGIS/issues

**Tienes que dar clic en el boton verde!**

<center>
<img src="https://user-images.githubusercontent.com/16768318/79680748-d5511000-81d8-11ea-9f89-44bd010adf69.png" width = 70%>
</center>